In [1]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [2]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [3]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [4]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [5]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [6]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [7]:
for epoch in range(1, 51):
    train(epoch)
    test()

C:\Users\lyeoni\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 544.540078
Train Epoch: 1 [10000/60000 (17%)]	Loss: 184.232109
Train Epoch: 1 [20000/60000 (33%)]	Loss: 162.313955
Train Epoch: 1 [30000/60000 (50%)]	Loss: 165.958750
Train Epoch: 1 [40000/60000 (67%)]	Loss: 159.636836
Train Epoch: 1 [50000/60000 (83%)]	Loss: 157.480146
====> Epoch: 1 Average loss: 178.0764
====> Test set loss: 162.4080
Train Epoch: 2 [0/60000 (0%)]	Loss: 165.543457
Train Epoch: 2 [10000/60000 (17%)]	Loss: 169.160371
Train Epoch: 2 [20000/60000 (33%)]	Loss: 156.650479
Train Epoch: 2 [30000/60000 (50%)]	Loss: 157.932402
Train Epoch: 2 [40000/60000 (67%)]	Loss: 149.316348
Train Epoch: 2 [50000/60000 (83%)]	Loss: 152.364521
====> Epoch: 2 Average loss: 157.7688
====> Test set loss: 154.8181
Train Epoch: 3 [0/60000 (0%)]	Loss: 156.378076
Train Epoch: 3 [10000/60000 (17%)]	Loss: 152.164893
Train Epoch: 3 [20000/60000 (33%)]	Loss: 149.615713
Train Epoch: 3 [30000/60000 (50%)]	Loss: 156.538242
Train Epoch: 3 [40000/60000 (67%)]	Loss: 159.81

====> Test set loss: 138.6256
Train Epoch: 44 [0/60000 (0%)]	Loss: 135.063555
Train Epoch: 44 [10000/60000 (17%)]	Loss: 133.560957
Train Epoch: 44 [20000/60000 (33%)]	Loss: 136.025967
Train Epoch: 44 [30000/60000 (50%)]	Loss: 131.632178
Train Epoch: 44 [40000/60000 (67%)]	Loss: 135.976533
Train Epoch: 44 [50000/60000 (83%)]	Loss: 144.262275
====> Epoch: 44 Average loss: 135.4606
====> Test set loss: 139.1682
Train Epoch: 45 [0/60000 (0%)]	Loss: 147.984717
Train Epoch: 45 [10000/60000 (17%)]	Loss: 139.047139
Train Epoch: 45 [20000/60000 (33%)]	Loss: 130.762754
Train Epoch: 45 [30000/60000 (50%)]	Loss: 136.663232
Train Epoch: 45 [40000/60000 (67%)]	Loss: 132.488057
Train Epoch: 45 [50000/60000 (83%)]	Loss: 131.014756
====> Epoch: 45 Average loss: 135.2890
====> Test set loss: 138.3633
Train Epoch: 46 [0/60000 (0%)]	Loss: 130.941514
Train Epoch: 46 [10000/60000 (17%)]	Loss: 145.028047
Train Epoch: 46 [20000/60000 (33%)]	Loss: 134.676533
Train Epoch: 46 [30000/60000 (50%)]	Loss: 138.489463

In [8]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')

C:\Users\lyeoni\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\torch\nn\functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
